In [1]:
import random
from functools import total_ordering

CREDIT_HOURS_LIMIT = 18
POPULATION_SIZE = 50
NUM_GENERATIONS = 100

In [2]:
class Course:
    def __init__(self, code, title, credit_hours, prerequisites=None):
        self.code = code
        self.title = title
        self.credit_hours = int(credit_hours)
        self.prerequisites = set() if prerequisites is None else set(prerequisites)

In [3]:
curriculum = {
    1: {
        "sc1201": Course("sc1201", "Applied Physics", "3", ""),
        "sc1001": Course("sc1001", "Calculus & Analytic Geometry", "3", ""),
        "hu1002": Course("hu1002", "English Composition & Comprehension", "3", ""),
        "cs1501": Course("cs1501", "Introduction to Information and Communication Technologies", "2", ""),
        "cs1001": Course("cs1001", "Programming Fundamentals", "4", " ")
    },
    2: {
        "hu1003": Course("hu1003", "Communication & Presentation Skills", "3", ""),
        "cs1502": Course("cs1502", "Digital Logic and Design", "4", ""),
        "hu1101": Course("hu1101", "Islamic Studies", "2", ""),
        "sc1002": Course("sc1002", "Multivariate Calculus", "2", "sc1001"),
        "cs2301": Course("cs2301", "Discrete Structures", "2", ""),
        "cs1002": Course("cs1002", "Programming Techniques", "2", "cs1001")
    },
    3: {
        "cs2503": Course("cs2503", "Computer Organization & Assembly Language", "4", ""),
        "cs2003": Course("cs2003", "Data Structure and Algorithms", "4", "cs1001"),
        "sc2003": Course("sc2003", "Differential Equations", "3", "sc1001"),
        "hu1102": Course("hu1102", "Pakistan Studies", "2", ""),
        "cs2004": Course("cs2004", "Object Oriented Programming", "4", "cs1001")
    },
    4: {
        "cs2201": Course("cs2201", "Introduction to Database Systems", "4", "cs2003"),
        "cs2504": Course("cs2504", "Operating Systems", "4", "cs2003"),
        "sc2004": Course("sc2004", "Probability and Statistics", "3", ""),
        "cs2101": Course("cs2101", "Software Engineering", "3", "")
    },
    5: {
        "cs4303": Course("cs4303", "Artificial Intelligence", "4", "cs2003"),
        "cs3005": Course("cs3005", "Design & Analysis of Algorithms", "3", "cs2003"),
        "cs3202": Course("cs3202", "Web Engineering", "4", "cs2003"),
        "cs3002": Course("cs3002", "Computer Networks", "3", "cs2003"),
        "cs3003": Course("cs3003", "Software Project Management", "3", "cs2101")
    },
    6: {
        "cs3701": Course("cs3701", "Computer Graphics", "4", "cs2004"),
        "cs4102": Course("cs4102", "Software Project-I", "4", "cs2004"),
        "cs3702": Course("cs3702", "Theory of Automata", "3", "cs2004"),
        "cs4001": Course("cs4001", "Digital Image Processing", "3", "cs2004"),
        "cs4602": Course("cs4602", "Data Warehousing & Data Mining", "3", "cs2201")
    },
    7: {
        "cs4701": Course("cs4701", "Introduction to Machine Learning", "3", "CS3005"),
        "cs4802": Course("cs4802", "Software Project-II", "3", "cs4102"),
        "cs4201": Course("cs4201", "Advanced Database Systems", "3", "cs2201"),
        "cs4302": Course("cs4302", "Software Quality Assurance", "3", "cs3003"),
        "cs4503": Course("cs4503", "Compiler Construction", "3", "cs3702"),
        "cs4004": Course("cs4004", "Parallel Computing", "3", "cs3002")
    },
    8: {
        "cs4801": Course("cs4801", "Final Year Project", "4", ""),
        "cs4902": Course("cs4902", "Software Project-III", "4", "cs4802"),
        "cs4502": Course("cs4502", "Advanced Operating Systems", "4", "cs2504"),
        "cs4603": Course("cs4603", "Human Computer Interaction", "3", ""),
        "cs4005": Course("cs4005", "Wireless Networks", "3", "cs3002")
    }
}

In [42]:
import random
from copy import deepcopy


In [43]:
def generate_chromosome(cur_semester, failed_courses, credit_hour_limit):
    chromosome = []
    for course_code, course in curriculum[cur_semester].items():
        if course_code not in failed_courses and sum(c.credit_hours for c in chromosome) + course.credit_hours <= credit_hour_limit:
            if all(prerequisite in [c.code for c in chromosome] for prerequisite in course.prerequisites):
                chromosome.append(course)
    return chromosome

def generate_population(cur_semester, failed_courses, credit_hour_limit, population_size):
    population = []
    for _ in range(population_size):
        chromosome = generate_chromosome(cur_semester, failed_courses, credit_hour_limit)
        population.append(chromosome)
    return population

def fitness(chromosome):
    return sum(course.credit_hours for course in chromosome)

def crossover(parent1, parent2):
    crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutation(chromosome, cur_semester, failed_courses, credit_hour_limit):
    mutated_chromosome = deepcopy(chromosome)
    for i in range(len(mutated_chromosome)):
        if random.random() < 0.2:
            mutated_chromosome.pop(i)
            available_courses = [course for course in curriculum[cur_semester].values()
                                 if course.code not in failed_courses and course not in mutated_chromosome
                                 and sum(c.credit_hours for c in mutated_chromosome) + course.credit_hours <= credit_hour_limit
                                 and all(prerequisite in [c.code for c in mutated_chromosome] for prerequisite in course.prerequisites)]
            if available_courses:
                mutated_chromosome.insert(i, random.choice(available_courses))
    return mutated_chromosome

def select_best_population(population, cur_semester, failed_courses, credit_hour_limit):
    best_population = []
    for chromosome in population:
        if sum(course.credit_hours for course in chromosome) <= credit_hour_limit:
            if all(prerequisite in [c.code for c in chromosome] for prerequisite in curriculum[cur_semester + 1]):
                best_population.append(chromosome)
    return best_population


def genetic_algorithm(cur_semester, failed_courses, credit_hour_limit, population_size, generations):
    population = generate_population(cur_semester, failed_courses, credit_hour_limit, population_size)

    for _ in range(generations):
        next_generation = []
        while len(next_generation) < population_size:
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, cur_semester, failed_courses, credit_hour_limit)
            mutated_child2 = mutation(child2, cur_semester, failed_courses, credit_hour_limit)
            next_generation.extend([mutated_child1, mutated_child2])

        population = select_best_population(next_generation, cur_semester, failed_courses, credit_hour_limit)
        if population:
            best_chromosome = max(population, key=fitness)
            return best_chromosome

    return None



In [44]:
current_semester = 3
failed_courses = ["cs2004", "sc2003"]
credit_hour_limit = 14
population_size = 50
generations = 100

best_chromosome = genetic_algorithm(current_semester, failed_courses, credit_hour_limit, population_size, generations)

if best_chromosome:
    print("Best Chromosome (New Plan):")
    for course in best_chromosome:
        print(course.code, course.title, course.credit_hours)
else:
    print("No valid plan found.")


IndexError: Cannot choose from an empty sequence

In [ ]:
# taking input from user
current_semester = int(input("Enter current semester: ")) # 4
failed_courses = input("Enter failed courses separated by space: ").split() # cs2003 cs2503 sc1002

In [7]:
# taking input from user
current_semester = int(input("Enter current semester: ")) # 4
failed_courses = input("Enter failed courses separated by space: ").split() # cs2003 cs2503 sc1002

# completed courses
completed_courses = []
for i in range(1, current_semester+1):
    completed_courses.extend(curriculum[i].keys())

#print("Completed courses: ", completed_courses)

# failed courses are not considered as completed courses
for course in failed_courses:
    if course in completed_courses:
        completed_courses.remove(course)

print("Completed courses after removing failed courses: ", completed_courses)

Completed courses after removing failed courses:  ['sc1201', 'sc1001', 'hu1002', 'cs1501', 'cs1001', 'hu1003', 'cs1502', 'hu1101', 'sc1002', 'cs2301', 'cs1002', 'cs2503', 'cs2003', 'sc2003', 'hu1102', 'cs2004', 'cs2201', 'cs2504', 'sc2004', 'cs4303', 'cs3005', 'cs3202', 'cs3002', 'cs3003']


In [9]:
# courses that can be taken in current semester
courses = []
for course in curriculum[current_semester].values():
    if course.code not in completed_courses:
        # now checking if the prerequisites of the course are completed
        if course != "" and course.prerequisites.issubset(completed_courses):
            courses.append(course)

print("Courses that can be taken in current semester: ")
for course in courses:
    print(course.code, course.title, course.credit_hours)        

Courses that can be taken in current semester: 


In [40]:
# now adding failed courses to the list of courses
for course in failed_courses:
    for semester in range(1, current_semester):
        if course in curriculum[semester].keys():
            courses.append(curriculum[semester][course])

# now we have all the courses that can be taken in current semester
for course in courses:
    print(course.code, course.title, course.credit_hours)

sc2004 Probability and Statistics 3
cs2101 Software Engineering 3
cs2003 Data Structure and Algorithms 4
cs2503 Computer Organization & Assembly Language 4
sc1002 Multivariate Calculus 2


In [41]:
# now to check if the credit hours limit is exceeded
credit_hours = 0
for course in courses:
    credit_hours += int(course.credit_hours)
    
if credit_hours > CREDIT_HOURS_LIMIT:
        random.shuffle(courses)
        courses.pop()   
        